 When you run this, it will download the perceiver model from github. Along with the train folder containing the `experiment.py` and `dataset.py`

* Folders can be seen on the left hand side of the screen 




In [ ]:
import tensorflow as tf

import os


resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.57.100.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.57.100.202:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
!pip install dm-haiku
!pip install einops

!mkdir /content/perceiver
!touch /content/perceiver/__init__.py
!wget -O /content/perceiver/io_processors.py https://raw.githubusercontent.com/deepmind/deepmind-research/master/perceiver/io_processors.py
!wget -O /content/perceiver/perceiver.py https://raw.githubusercontent.com/deepmind/deepmind-research/master/perceiver/perceiver.py
!wget -O /content/perceiver/position_encoding.py https://raw.githubusercontent.com/deepmind/deepmind-research/master/perceiver/position_encoding.py
!mkdir /content/perceiver/train 
!wget -O /content/perceiver/train/autoaugment.py https://raw.githubusercontent.com/deepmind/deepmind-research/master/perceiver/train/autoaugment.py
!wget -O /content/perceiver/train/dataset.py    https://raw.githubusercontent.com/deepmind/deepmind-research/master/perceiver/train/dataset.py
!wget -O /content/perceiver/experiment.py https://raw.githubusercontent.com/deepmind/deepmind-research/master/perceiver/train/experiment.py
!wget -O /content/perceiver/train/utils.py  https://raw.githubusercontent.com/deepmind/deepmind-research/master/perceiver/train/utils.py

!pip install jaxline
!pip install optax
!pip install tensorflow-addons

%cd perceiver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 342 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-07-24 16:11:00--  https://raw.githubusercontent.com/deepmind/deepmind-research/master/perceiver/io_processors.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29359 (29K) [text/plain]
Saving to: ‘/content/perceiver/io_processors.py’

/content/perceiver/ 100%[===================>]  28.67K  --.-KB/s    in 0.002s  

2022-07-24 16:11:00 (16.8 MB/s) - ‘/content/perceiver/io_processors.py’ saved [29359/29359]

--2022-07-24 16:11:00--  https://raw.githubusercontent.com/deepmind/deepmind-research/master/per

In [ ]:
datafiles = ["/content/perceiver/perceiver.py", '/content/perceiver/train/dataset.py', '/content/perceiver/experiment.py' ]

def remove_from_per(filename):
  # Read in the file
  with open(filename, 'r') as file :
    filedata = file.read()

  # Replace the target string
  filedata = filedata.replace('from perceiver ', '')
  if ('experiment.py' in filename):
    filedata = filedata.replace('PerceiverEncoder', 'perceiver.PerceiverEncoder')
    filedata = filedata.replace('ClassificationDecoder', 'perceiver.ClassificationDecoder')
    filedata = filedata.replace('Perceiver', 'perceiver.Perceiver')

  # Write the file out again
  with open(filename, 'w') as file:
    file.write(filedata)


def remove_from_per_dot(filename):
  # Read in the file
  with open(filename, 'r') as file :
    filedata = file.read()

  # Replace the target string
  filedata = filedata.replace('perceiver.', '')

  # Write the file out again
  with open(filename, 'w') as file:
    file.write(filedata)

def remove_label(filename):
  # Read in the file
  with open(filename, 'r') as file :
    filedata = file.read()

  # Replace the target string
  filedata = filedata.replace('host_label', 'host_id')
  filedata = filedata.replace('imagenet2012:5.*.*', 'oxford_flowers102')
  filedata = filedata.replace('1281167', '1020')
  filedata = filedata.replace('1271167','1020')
  filedata = filedata.replace('10000', '1020')
  filedata = filedata.replace('50000', '6149')

  # Write the file out again
  with open(filename, 'w') as file:
    file.write(filedata)

remove_from_per(datafiles[0])
remove_from_per_dot(datafiles[1])

remove_from_per_dot(datafiles[2])
remove_from_per(datafiles[2])

remove_label('/content/perceiver/train/dataset.py')


In [ ]:
# @title Experiment.py 

import functools
from typing import Generator, Mapping, Text, Tuple

from absl import app
from absl import flags
from absl import logging
import haiku as hk
import jax
import jax.numpy as jnp
from jaxline import base_config
from jaxline import experiment
from jaxline import platform
from jaxline import utils as jl_utils
from ml_collections import config_dict
import numpy as np
import optax


import io_processors
import perceiver
from train import dataset
from train import utils

FLAGS = flags.FLAGS

OptState = Tuple[optax.TraceState, optax.ScaleByScheduleState, optax.ScaleState]
Scalars = Mapping[Text, jnp.ndarray]


N_TRAIN_EXAMPLES = dataset.Split.TRAIN_AND_VALID.num_examples
N_CLASSES = 600
# Only local/debug parameters are supported out of the box.
# To use the scaled-up hyperparameters, please adapt this script to your
# training setup and set this flag to False
IS_LOCAL = True
NUM_FRAMES = 16
SAMPLES_PER_PATCH = 16
NUM_CLASSES = 600
IMG_SZ = 56


def get_training_steps(batch_size, n_epochs):
  return (N_TRAIN_EXAMPLES * n_epochs) // batch_size


def get_config():
  """Return config object for training."""
  use_debug_settings = IS_LOCAL
  config = base_config.get_base_config()

  # Experiment config.
  local_batch_size = 2
  # Modify this to adapt to your custom distributed learning setup
  num_devices = 1
  config.train_batch_size = local_batch_size * num_devices
  config.n_epochs = 110

  def _default_or_debug(default_value, debug_value):
    return debug_value if use_debug_settings else default_value

  n_train_examples = N_TRAIN_EXAMPLES
  num_classes = N_CLASSES

  config.experiment_kwargs = config_dict.ConfigDict(
      dict(
          config=dict(
              optimizer=dict(
                  base_lr=5e-4,
                  max_norm=10.0,  # < 0 to turn off.
                  schedule_type='constant_cosine',
                  weight_decay=1e-1,
                  decay_pos_embs=True,
                  scale_by_batch=True,
                  cosine_decay_kwargs=dict(
                      init_value=0.0,
                      warmup_epochs=0,
                      end_value=0.0,
                  ),
                  step_decay_kwargs=dict(
                      decay_boundaries=[0.5, 0.8, 0.95],
                      decay_rate=0.1,
                  ),
                  constant_cosine_decay_kwargs=dict(
                      constant_fraction=0.5,
                      end_value=0.0,
                  ),
                  optimizer='lamb',
                  # Optimizer-specific kwargs:
                  adam_kwargs=dict(
                      b1=0.9,
                      b2=0.999,
                      eps=1e-8,
                  ),
                  lamb_kwargs=dict(
                      b1=0.9,
                      b2=0.999,
                      eps=1e-6,
                  ),
              ),
          

              model=dict(
                  perceiver_kwargs=dict(
                      encoder=dict(
                        num_self_attends_per_block=8,
                        # Weights won't be shared if num_blocks is set to 1.
                        num_blocks=1,
                        z_index_dim=28*28*1,
                        num_z_channels=512,
                        num_cross_attend_heads=1,
                        num_self_attend_heads=8,
                        cross_attend_widening_factor=1,
                        self_attend_widening_factor=1,
                        dropout_prob=0.0,
                        z_pos_enc_init_scale=0.02,
                        cross_attention_shape_for_attn='kv',
                        name='encoder'
                        ),

                      decoder=dict(
                        # Autoencoding, don't pass inputs to the queries.
                        concat_preprocessed_input=False,                        
                        # Modality specific decoders are used ONLY to generate queries.
                        # All modalties are decoded together using a unified decoder.                       
                        num_outputs=None,
                        output_num_channels=512,
                        use_query_residual=False,
                      ),
                  ),
              ),
            
              training=dict(
                  inputs_per_epoch=n_train_examples,
                  label_smoothing=0.1,
                  n_epochs=config.get_oneway_ref('n_epochs'),
                  batch_size=config.get_oneway_ref('train_batch_size')
              ),
              data=dict(
                  num_classes=num_classes,
                  # Run on smaller inputs to debug.
                  im_dim=_default_or_debug(224, 32),
                  augmentation=dict(
                      # Typical randaug params:
                      # num_layers in [1, 3]
                      # magnitude in [5, 30]
                      # Set randaugment to None to disable.
                      # randaugment=dict(
                      #     num_layers=4,
                      #     magnitude=5),
                      # cutmix=True,
                      # Mixup alpha should be in [0, 1].
                      # Set to None to disable.
                      # mixup_alpha=0.2,
                  ),
                  ),
              evaluation=dict(
                  subset='test',
                  batch_size=2,
              ),
          )
      )
  )

  # Training loop config.
  config.training_steps = get_training_steps(
      config.get_oneway_ref('train_batch_size'),
      config.get_oneway_ref('n_epochs'))
  config.log_train_data_interval = 60
  config.log_tensors_interval = 60
  config.save_checkpoint_interval = 300
  config.eval_specific_checkpoint_dir = ''
  config.best_model_eval_metric = 'eval_top_1_acc'
  config.checkpoint_dir = '/tmp/perceiver_imagnet_checkpoints'
  config.train_checkpoint_all_hosts = False

  # Prevents accidentally setting keys that aren't recognized (e.g. in tests).
  config.lock()

  return config


class Experiment(experiment.AbstractExperiment):
  """ImageNet experiment."""

  # A map from object properties that will be checkpointed to their name
  # in a checkpoint. Currently we assume that these are all sharded
  # device arrays.
  CHECKPOINT_ATTRS = {
      '_params': 'params',
      '_state': 'state',
      '_opt_state': 'opt_state',
  }

  def __init__(self, mode, init_rng, config):
    """Initializes experiment."""

    super(Experiment, self).__init__(mode=mode, init_rng=init_rng)

    self.mode = mode
    self.init_rng = init_rng
    self.config = config

    # Checkpointed experiment state.
    self._params = None
    self._state = None
    self._opt_state = None

    # Input pipelines.
    self._train_input = None
    self._eval_input = None

    self.forward = hk.transform_with_state(self._forward_fn)

    # NOTE: We "donate" the `params, state, opt_state` arguments which allows
    # JAX (on some backends) to reuse the device memory associated with these
    # inputs to store the outputs of our function (which also start with
    # `params, state, opt_state`).
    self._update_func = jax.pmap(self._update_func, axis_name='i',
                                 donate_argnums=(0, 1, 2))
    self._eval_batch = jax.jit(self._eval_batch)

  def _forward_fn(
      self,
      inputs: dataset.Batch,
      is_training: bool,
      subsampling
  ) -> jnp.ndarray:

    inputs = inputs['images']
    
    subsampled_index_dims = {      
        'image': subsampling['image'].shape[0],
        'label': 1,
    }
    
      
    
    perceiver_kwargs = self.config.model.perceiver_kwargs
    output_postprocessor = io_processors.MultimodalPostprocessor(  modalities={                            
            'image': io_processors.ProjectionPostprocessor(
                num_outputs=3),
            'label': io_processors.ClassificationPostprocessor(
                num_classes=NUM_CLASSES),
        })
    input_preprocessor = io_processors.MultimodalPreprocessor(
        min_padding_size=4,        
        modalities={      
            'image': io_processors.ImagePreprocessor(
                position_encoding_type= 'fourier',
                fourier_position_encoding_kwargs=dict(
                    num_bands=32,
                    max_resolution=(NUM_FRAMES, IMG_SZ, IMG_SZ),
                    sine_only=False,
                    concat_pos=True,
                ),
                n_extra_pos_mlp=0,
                prep_type='patches',
                spatial_downsample=4,
                temporal_downsample=1),
            'label': io_processors.OneHotPreprocessor(),
        },
        mask_probs={'image': 0.0, 'audio': 0.0, 'label': 1.0},
        )
    encoder = perceiver.PerceiverEncoder(**perceiver_kwargs['encoder'])
    decoder = perceiver.MultimodalDecoder(
        subsampled_index_dims=subsampled_index_dims,
        modalities={
                'image': perceiver.BasicVideoAutoencodingDecoder(
                    concat_preprocessed_input=False,
                    subsampled_index_dims=subsampling['image'],
                    output_shape=inputs.shape[:4],
                    num_z_channels=1024,
                    output_num_channels=512,
                    use_query_residual=False,
                    position_encoding_type='fourier',
                    fourier_position_encoding_kwargs=dict(
                      num_bands=32,
                      max_resolution=(NUM_FRAMES, IMG_SZ, IMG_SZ),
                      sine_only=False,
                      concat_pos=True,
                    ),   
                ),
                'label': perceiver.ClassificationDecoder(
                    # Autoencoding, don't pass inputs to the queries.
                    concat_preprocessed_input=False,
                    num_classes=NUM_CLASSES,
                    num_z_channels=1024,
                    use_query_residual=False,
                    position_encoding_type='trainable',
                    trainable_position_encoding_kwargs=dict(
                        num_channels=1024,
                        init_scale=0.02,
                    ),
                ),
            },
        **perceiver_kwargs['decoder'])
    
    model = perceiver.Perceiver(
      input_preprocessor=input_preprocessor,
      encoder=encoder,
      decoder=decoder,
      output_postprocessor=output_postprocessor)

    return model({'image': inputs,                
                'label': np.zeros((inputs.shape[0], 600))},
               is_training=is_training, subsampled_output_points=subsampling)

  #  _             _
  # | |_ _ __ __ _(_)_ __
  # | __| '__/ _` | | '_ \
  # | |_| | | (_| | | | | |
  #  \__|_|  \__,_|_|_| |_|
  #

  def step(self, global_step: int, rng: jnp.ndarray,
           *unused_args, **unused_kwargs):
    """See base class."""

    if self._train_input is None:
      self._initialize_train()

    inputs = next(self._train_input)

    self._params, self._state, self._opt_state, scalars = (
        self._update_func(
            self._params, self._state, self._opt_state, inputs, rng, global_step
            ))

    scalars = jl_utils.get_first(scalars)
    return scalars

  def _initialize_train(self):
    self._train_input = jl_utils.py_prefetch(self._build_train_input)

    total_batch_size = self.config.training.batch_size
    steps_per_epoch = (
        self.config.training.inputs_per_epoch / self.config.training.batch_size)
    total_steps = self.config.training.n_epochs * steps_per_epoch
    # Scale by the (negative) learning rate.
    self._lr_schedule = utils.get_learning_rate_schedule(
        total_batch_size, steps_per_epoch, total_steps, self.config.optimizer)

    self._optimizer = utils.make_optimizer(
        self.config.optimizer,
        self._lr_schedule)

    # Check we haven't already restored params
    if self._params is None:
      logging.info('Initializing parameters.')

      inputs = next(self._train_input)
      nchunks = 128
      image_chunk_size = np.prod(inputs['images'].shape[1:-1]) // nchunks
      subsampling = {
            'image': jnp.arange(
                image_chunk_size * 0 , image_chunk_size * ( 1)),
            
            'label': None,
        }
    
      init_net = jax.pmap(lambda *a: self.forward.init(*a,  subsampling=subsampling, is_training=True))
      init_opt = jax.pmap(self._optimizer.init)

      # Init uses the same RNG key on all hosts+devices to ensure everyone
      # computes the same initial state.
      init_rng = jl_utils.bcast_local_devices(self.init_rng)

      self._params, self._state = init_net(init_rng, inputs)
      self._opt_state = init_opt(self._params)

  def _load_data(self, split, is_training, batch_dims):
    """Wrapper for dataset loading."""

    return dataset.load(
        split=split,
        is_training=is_training,
        batch_dims=batch_dims,
        im_dim=self.config.data.im_dim,
        augmentation_settings=self.config.data.augmentation,
        )

  def _build_train_input(self) -> Generator[dataset.Batch, None, None]:
    """See base class."""
    num_devices = jax.device_count()
    global_batch_size = self.config.training.batch_size
    per_device_batch_size, ragged = divmod(global_batch_size, num_devices)

    if ragged:
      raise ValueError(
          f'Global batch size {global_batch_size} must be divisible by '
          f'num devices {num_devices}')

    split = dataset.Split.TRAIN_AND_VALID

    return self._load_data(
        split=split,
        is_training=True,
        batch_dims=[jax.local_device_count(), per_device_batch_size])

  def _one_hot(self, value):
    """One-hot encoding potentially over a sequence of labels."""
    y = jax.nn.one_hot(value, self.config.data.num_classes)
    return y

  def _loss_fn(
      self,
      params: hk.Params,
      state: hk.State,
      inputs: dataset.Batch,
      rng: jnp.ndarray,
  ) -> Tuple[jnp.ndarray, Tuple[Scalars, hk.State]]:
    nchunks = 128
    reconstruction = {}
    
    for chunk_idx in range(nchunks):
        image_chunk_size = np.prod(inputs['images'].shape[1:-1]) // nchunks
        
        subsampling = {
            'image': jnp.arange(
                image_chunk_size * chunk_idx, image_chunk_size * (chunk_idx + 1)),
           
            'label': None,
        }
        
        output, state = self.forward.apply(
            params, state, rng, inputs, subsampling=subsampling, is_training=True)
    
        reconstruction['label'] = output['label']
        if 'image' not in reconstruction:
            reconstruction['image'] = output['image']
            
        else:
            reconstruction['image'] = jnp.concatenate(
                [reconstruction['image'], output['image']], axis=1)
          
            
        reconstruction['image'] = jnp.reshape(reconstruction['image'], inputs['images'].shape)

    # label = self._one_hot(inputs['labels'])
    # Handle cutmix/mixup label mixing:
    # if 'mix_labels' in inputs:
    #   logging.info('Using mixup or cutmix!')
    #   mix_label = self._one_hot(inputs['mix_labels'])
    #   mix_ratio = inputs['ratio'][:, None]
    #   label = mix_ratio * label + (1. - mix_ratio) * mix_label

    # # Apply label-smoothing to one-hot labels.
    # label_smoothing = self.config.training.label_smoothing
    # if not (label_smoothing >= 0. and label_smoothing < 1.):
    #   raise ValueError(
    #       f"'label_smoothing is {label_smoothing} and should be in [0, 1)")
    # if label_smoothing > 0:
    #   smooth_positives = 1. - label_smoothing
    #   smooth_negatives = label_smoothing / self.config.data.num_classes
    #   label = smooth_positives * label + smooth_negatives


######## CHANGE!!!!!!!!!!!!!!! ###################
    loss_w_batch = utils.softmax_cross_entropy(output, label)
    loss = jnp.mean(loss_w_batch, dtype=loss_w_batch.dtype)
    scaled_loss = loss / jax.device_count()

    metrics = utils.topk_correct(output, inputs['labels'], prefix='')
    metrics = jax.tree_map(jnp.mean, metrics)

    top_1_acc = metrics['top_1_acc']
    top_5_acc = metrics['top_5_acc']

    loss_scalars = dict(
        loss=loss,
        top_1_acc=top_1_acc,
        top_5_acc=top_5_acc,
    )

    return scaled_loss, (loss_scalars, state)

  def _update_func(
      self,
      params: hk.Params,
      state: hk.State,
      opt_state: OptState,
      inputs: dataset.Batch,
      rng: jnp.ndarray,
      global_step: int,
  ) -> Tuple[hk.Params, hk.State, OptState, Scalars]:
    """Applies an update to parameters and returns new state."""
    # This function computes the gradient of the first output of loss_fn and
    # passes through the other arguments unchanged.
    grad_loss_fn = jax.grad(self._loss_fn, has_aux=True)
    scaled_grads, (loss_scalars, state) = grad_loss_fn(
        params, state, inputs, rng)
    grads = jax.lax.psum(scaled_grads, axis_name='i')

    # Grab the learning rate to log before performing the step.
    learning_rate = self._lr_schedule(global_step)

    # Compute and apply updates via our optimizer.
    updates, opt_state = self._optimizer.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)

    n_params = 0
    for k in params.keys():
      for l in params[k]:
        n_params = n_params + np.prod(params[k][l].shape)

    # Scalars to log (note: we log the mean across all hosts/devices).
    scalars = {'learning_rate': learning_rate,
               'n_params (M)': float(n_params/1e6),
               'global_gradient_norm': optax.global_norm(grads)}
    loss_scalars = {f'train_{k}': v for k, v in loss_scalars.items()}
    scalars.update(loss_scalars)
    scalars = jax.lax.pmean(scalars, axis_name='i')

    return params, state, opt_state, scalars

  #                  _
  #   _____   ____ _| |
  #  / _ \ \ / / _` | |
  # |  __/\ V / (_| | |
  #  \___| \_/ \__,_|_|
  #

  def evaluate(self, global_step, rng, **unused_args):
    """See base class."""
    global_step = np.array(jl_utils.get_first(global_step))
    scalars = jax.device_get(self._eval_epoch(jl_utils.get_first(rng)))

    logging.info('[Step %d] Eval scalars: %s', global_step, scalars)
    return scalars

  def _eval_batch(
      self,
      params: hk.Params,
      state: hk.State,
      inputs: dataset.Batch,
      rng: jnp.ndarray,
  ) -> Scalars:
    """Evaluates a batch."""
    nchunks = 128
    reconstruction = {}
    for chunk_idx in range(nchunks):
        image_chunk_size = np.prod(inputs['images'].shape[1:-1]) // nchunks
        
        subsampling = {
            'image': jnp.arange(
                image_chunk_size * chunk_idx, image_chunk_size * (chunk_idx + 1)),
            
            'label': None,
        }
        output, _ = self.forward.apply(
            params, state, rng, inputs, subsampling, is_training=False)
        reconstruction['label'] = output['label']
        if 'image' not in reconstruction:
            reconstruction['image'] = output['image']
            # reconstruction['audio'] = output['audio']
        else:
            reconstruction['image'] = jnp.concatenate(
                [reconstruction['image'], output['image']], axis=1)
            # reconstruction['audio'] = jnp.concatenate(
            #     [reconstruction['audio'], output['audio']], axis=1)
        
    reconstruction['image'] = jnp.reshape(reconstruction['image'], inputs['images'].shape)
    
    

    
    ### CHANGE##############

    labels = self._one_hot(inputs['labels'])
    loss = utils.softmax_cross_entropy(output, labels)

    metrics = utils.topk_correct(output, inputs['labels'], prefix='')
    metrics = jax.tree_map(jnp.mean, metrics)
    top_1_acc = metrics['top_1_acc']
    top_5_acc = metrics['top_5_acc']

    bs = output.shape[0]

    top_1_acc = jnp.expand_dims(top_1_acc, axis=0) * bs
    top_5_acc = jnp.expand_dims(top_5_acc, axis=0) * bs

    # NOTE: Returned values will be summed and finally divided by num_samples.
    return {
        'eval_loss': loss,
        'eval_top_1_acc': top_1_acc, 'eval_top_5_acc': top_5_acc}

  def _build_eval_input(self) -> Generator[dataset.Batch, None, None]:
    split = dataset.Split.from_string(self.config.evaluation.subset)

    return self._load_data(
        split=split,
        is_training=False,
        batch_dims=[self.config.evaluation.batch_size])

  def _eval_epoch(self, rng):
    """Evaluates an epoch."""
    num_samples = 0.
    summed_scalars = None

    params = jl_utils.get_first(self._params)
    state = jl_utils.get_first(self._state)

    for inputs in self._build_eval_input():
      num_samples += inputs['labels'].shape[0]
      scalars = self._eval_batch(params, state, inputs, rng)

      # Accumulate the sum of scalars for each step.
      scalars = jax.tree_map(lambda x: jnp.sum(x, axis=0), scalars)
      if summed_scalars is None:
        summed_scalars = scalars
      else:
        summed_scalars = jax.tree_multimap(jnp.add, summed_scalars, scalars)

    mean_scalars = jax.tree_map(lambda x: x / num_samples, summed_scalars)
    return mean_scalars


if __name__ == '__main__':
  flags.mark_flag_as_required('config')
  app.run(functools.partial(platform.main, Experiment))


In [ ]:
# @title dataset.py
import enum
from typing import Any, Generator, Mapping, Optional, Sequence, Text, Tuple
import os
import jax
import random
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

from train import autoaugment


Batch = Mapping[Text, np.ndarray]
MEAN_RGB = (0.485 * 255, 0.456 * 255, 0.406 * 255)
STDDEV_RGB = (0.229 * 255, 0.224 * 255, 0.225 * 255)
AUTOTUNE = tf.data.experimental.AUTOTUNE

INPUT_DIM = 224  # The number of pixels in the image resize.


class Split(enum.Enum):
  """ImageNet dataset split."""
  TRAIN = 1
  TRAIN_AND_VALID = 2
  VALID = 3
  TEST = 4

  @classmethod
  def from_string(cls, name: Text) -> 'Split':
    return {'TRAIN': Split.TRAIN, 'TRAIN_AND_VALID': Split.TRAIN_AND_VALID,
            'VALID': Split.VALID, 'VALIDATION': Split.VALID,
            'TEST': Split.TEST}[name.upper()]

  @property
  def num_examples(self):
    return {Split.TRAIN_AND_VALID: 40, Split.TRAIN: 40,
            Split.VALID: 40, Split.TEST: 40}[self]


def load(
    split: Split,
    *,
    is_training: bool,
    # batch_dims should be:
    # [device_count, per_device_batch_size] or [total_batch_size]
    batch_dims: Sequence[int],
    augmentation_settings: Mapping[str, Any],
    # The shape to which images are resized.
    im_dim: int = INPUT_DIM,
    threadpool_size: int = 48,
    max_intra_op_parallelism: int = 1,
) -> Generator[Batch, None, None]:
  """Loads the given split of the dataset."""
  # start, end = _shard(split, jax.host_id(), jax.process_count())
  
  im_size = (im_dim, im_dim)

  total_batch_size = np.prod(batch_dims)
  full_list = []
  if (split == Split.TRAIN or split == Split.TRAIN_AND_VALID ):
    path = '/content/dataset/train'
    filenames_list = os.listdir(path)    
  elif (split == Split.VALID):
      path = '/content/dataset/valid'
      filenames_list  = os.listdir(path)      
  else:
      path = '/content/dataset/test'
      filenames_list  = os.listdir(path)

  for file in filenames_list:
    full_list.append(os.path.join(path,file))

      
  ds = dataset_numpy(full_list)
  options = tf.data.Options()
  options.threading.private_threadpool_size = threadpool_size
  options.threading.max_intra_op_parallelism = (
      max_intra_op_parallelism)
  options.experimental_optimization.map_parallelization = True
  if is_training:
    options.experimental_deterministic = False
  ds = ds.with_options(options)

  if is_training:
    if jax.process_count() > 1:
      # Only cache if we are reading a subset of the dataset.
      ds = ds.cache()
    ds = ds.repeat()
    ds = ds.shuffle(buffer_size=10 * total_batch_size, seed=0)

  else:
    if split.num_examples % total_batch_size != 0:
      raise ValueError(f'Test/vallabel must be divisible by {total_batch_size}')

  def crop_augment_preprocess(example):
    image = example #_preprocess_image(example['image'], is_training, im_size, augmentation_settings)
    label_init = random.randint(0, 600)
    label = tf.cast(label_init, tf.int32)
    out = {'images': image, 'labels': label}
    return out

  ds = ds.map(crop_augment_preprocess, num_parallel_calls=AUTOTUNE)

  for batch_size in reversed(batch_dims):
    ds = ds.batch(batch_size)

  ds = ds.prefetch(AUTOTUNE)

  yield from tfds.as_numpy(ds)


def dataset_numpy(filenames_list):
    if filenames_list :
        # initialize train dataset
        train_dataset = np.expand_dims(np.load(filenames_list[0]),axis=0 )
        print("init shape: ", train_dataset.shape)
        ds = tf.data.Dataset.from_tensor_slices(train_dataset)    
        print("read frist file: ", ds)
        # concatenate with the remaining files  
        for file in filenames_list[1:]: 
            read_data = np.expand_dims(np.load(file),axis=0 )

            add_ds = tf.data.Dataset.from_tensor_slices((read_data))
            ds = ds.concatenate(add_ds)
        return ds 
    else:
        print("empty list")
    


In [ ]:
# !pip uninstall tensorflow-io
!pip install tensorflow-gpu
!pip install --no-deps tensorflow-io
!pip install jaxline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.1 kB/s 
     |████████████████████████████████| 1.6 MB 39.7 MB/s 
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 438 kB 40.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.9 MB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Dummy dataset


In [ ]:
import os
import numpy as np 
paths = ['/content/dataset/train','/content/dataset/test', '/content/dataset/valid']
if (not os.path.exists('/content/dataset')):
  os.mkdir('/content/dataset')
for subdir in paths:
  if (not os.path.exists(subdir)):
    os.mkdir(subdir)
  for i in range(40):
    video = np.random.rand(16,224,224,3)
    path = os.path.join(subdir, str(i))
    np.save(path, video)


In [ ]:
t = np.load('/content/dataset/test/0.npy')
t.shape

(16, 224, 224, 3)

#  Pre-trained Autoencoder



[Colab original code](https://colab.research.google.com/github/deepmind/deepmind_research/blob/master/perceiver/colabs/video_autoencoding.ipynb#scrollTo=uxeP5yit7hJg)

[Github for perceiver - all files and examples](https://github.com/deepmind/deepmind-research/tree/master/perceiver)





In [ ]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [ ]:
!pip install -U jax[cuda11_cudnn82] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.htm

In [ ]:
import jax
print("jax backend {}".format(jax.lib.xla_bridge.get_backend().platform))
jax.devices()

jax backend cpu


[CpuDevice(id=0)]

In [ ]:
print("All GPU devices: ", tf.config.list_logical_devices('GPU'))
print("All TPU devices: ", tf.config.list_logical_devices('TPU'))

All GPU devices:  []
All TPU devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
!pip install -U jaxlib==0.3.14+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     |████████████████████████████████| 161.9 MB 64 kB/s 
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.3.14+cuda11.cudnn805
    Uninstalling jaxlib-0.3.14+cuda11.cudnn805:
      Successfully uninstalled jaxlib-0.3.14+cuda11.cudnn805


# RUN 


In [ ]:
!python /content/perceiver/experiment.py --config=/content/perceiver/experiment.py  --logtostderr

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
2022-07-24 16:19:23.605808: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]
W0724 16:19:55.372099 140215549192064 tpu_strategy_uti

In [ ]:
!zip -r /content/perceiver_working.zip /content/perceiver

In [ ]:
import jax
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

ds = tfds.load(name="mnist", split="train")
ds_numpy = tfds.as_numpy(ds)  # Convert `tf.data.Dataset` to Python generator
for ex in ds_numpy:
  # `{'image': np.array(shape=(28, 28, 1)), 'labels': np.array(shape=())}`
  # print(ex)
  print(ex.keys())
  
  print(ex.values())


In [ ]:
ds

<PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [ ]:
import enum
from typing import Any, Generator, Mapping, Optional, Sequence, Text, Tuple
import os
import jax
import random
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

Batch = Mapping[Text, np.ndarray]
AUTOTUNE = tf.data.experimental.AUTOTUNE

def load(
    split: str,
    *,
    is_training: bool,
    # batch_dims should be:
    # [device_count, per_device_batch_size] or [total_batch_size]
    batch_dims: Sequence[int],   
    # The shape to which images are resized.
    # im_dim: int = INPUT_DIM,
    threadpool_size: int = 48,
    max_intra_op_parallelism: int = 1,
) -> Generator[Batch, None, None]:
  """Loads the given split of the dataset."""
  # start, end = _shard(split, jax.host_id(), jax.process_count())
  
  # im_size = (16,im_dim, im_dim)

  total_batch_size = np.prod(batch_dims)
  full_list = []
  if (split == "train"  ):
    path = '/content/dataset_img/train_img'
    filenames_list = os.listdir(path)    
  elif (split == "VALID"):
      path = '/content/dataset_img/valid_img'
      filenames_list  = os.listdir(path)      
  else:
      path = '/content/dataset/test'
      filenames_list  = os.listdir(path)

  for file in filenames_list:
    full_list.append(os.path.join(path,file))

      
  ds = dataset_numpy(full_list)
  options = tf.data.Options()
  options.threading.private_threadpool_size = threadpool_size
  options.threading.max_intra_op_parallelism = (
      max_intra_op_parallelism)
  options.experimental_optimization.map_parallelization = True
  if is_training:
    options.experimental_deterministic = False
  ds = ds.with_options(options)

  if is_training:
    if jax.process_count() > 1:
      # Only cache if we are reading a subset of the dataset.
      ds = ds.cache()
    ds = ds.repeat()
    ds = ds.shuffle(buffer_size=10 * total_batch_size, seed=0)

  else:
    if split.num_examples % total_batch_size != 0:
      raise ValueError(f'Test/vallabel must be divisible by {total_batch_size}')

  def crop_augment_preprocess(example):
    print("shape: ", example.shape)
    image = example #_preprocess_image(example['image'], is_training, im_size, augmentation_settings)
    label_init = random.randint(0, 600)
    label = tf.cast(label_init, tf.int32)
    out = {'images': image, 'labels': label}
    return out

  ds = ds.map(crop_augment_preprocess, num_parallel_calls=AUTOTUNE)

  for batch_size in reversed(batch_dims):
    ds = ds.batch(batch_size)
  print(ds)
  ds = ds.prefetch(AUTOTUNE)
  
  yield from tfds.as_numpy(ds)


def dataset_numpy(filenames_list):
    if filenames_list :
        # initialize train dataset
        train_dataset = np.expand_dims(np.load(filenames_list[0]),axis=0 )
        print("init shape: ", train_dataset.shape)
        ds = tf.data.Dataset.from_tensor_slices(train_dataset)    
        print("read frist file: ", ds)
        # concatenate with the remaining files  
        for file in filenames_list[1:]: 
            read_data = np.expand_dims(np.load(file),axis=0 )

            add_ds = tf.data.Dataset.from_tensor_slices((read_data))
            ds = ds.concatenate(add_ds)
        return ds 
    else:
        print("empty list")
     

In [ ]:
g = load(split='train', is_training=True,batch_dims=[32])
for gen in g:
  print(gen['images'].shape) 
  break

init shape:  (1, 224, 224, 3)
read frist file:  <TensorSliceDataset element_spec=TensorSpec(shape=(224, 224, 3), dtype=tf.float64, name=None)>
shape:  (224, 224, 3)
<BatchDataset element_spec={'images': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float64, name=None), 'labels': TensorSpec(shape=(None,), dtype=tf.int32, name=None)}>
(32, 224, 224, 3)


In [ ]:
!rm -rf /content/dataset/valid_img
